In [ ]:
# !pip install opencv-python numpy ultralytics filterpy torch torchvision
# !pip install yt-dlp

In [3]:
import requests

url = "https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt"
response = requests.get(url, stream=True)

with open("yolov8n.pt", "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print("Download complete!")

Download complete!


In [7]:
import subprocess


video_url_1 = "https://www.youtube.com/shorts/MwxxofLNpTg"
video_url_2 = "https://www.youtube.com/watch?v=HYSdFHQDDcs&t=30s"

subprocess.run(["yt-dlp", "-f", "bestvideo+bestaudio/best", video_url_1])
subprocess.run(["yt-dlp", "-f", "bestvideo+bestaudio/best", video_url_2])

[youtube] Extracting URL: https://www.youtube.com/shorts/MwxxofLNpTg
[youtube] MwxxofLNpTg: Downloading webpage
[youtube] MwxxofLNpTg: Downloading tv client config
[youtube] MwxxofLNpTg: Downloading player 6b3caec8
[youtube] MwxxofLNpTg: Downloading tv player API JSON
[youtube] MwxxofLNpTg: Downloading ios player API JSON
[youtube] MwxxofLNpTg: Downloading m3u8 information
[info] MwxxofLNpTg: Downloading 1 format(s): 247+251


[download] Destination: Active Track - Near Crash ｜ DJI Mini 4 Pro #dji #mini4 #followme #drone #activetrack #wildtrak [MwxxofLNpTg].f247.webm
[download] 100% of    4.03MiB in 00:00:00 at 12.46MiB/s    
[download] Destination: Active Track - Near Crash ｜ DJI Mini 4 Pro #dji #mini4 #followme #drone #activetrack #wildtrak [MwxxofLNpTg].f251.webm
[download] 100% of  228.36KiB in 00:00:00 at 5.32MiB/s     
[youtube] Extracting URL: https://www.youtube.com/watch?v=HYSdFHQDDcs&t=30s
[youtube] HYSdFHQDDcs: Downloading webpage
[youtube] HYSdFHQDDcs: Downloading tv client config
[youtube] HYSdFHQDDcs: Downloading player 74e4bb46
[youtube] HYSdFHQDDcs: Downloading tv player API JSON
[youtube] HYSdFHQDDcs: Downloading ios player API JSON
[youtube] HYSdFHQDDcs: Downloading m3u8 information
[info] HYSdFHQDDcs: Downloading 1 format(s): 136+251


[download] Destination: Traffic Count [HYSdFHQDDcs].f136.mp4
[download] 100% of    4.72MiB in 00:00:00 at 19.26MiB/s  
[download] Destination: Traffic Count [HYSdFHQDDcs].f251.webm
[download] 100% of   13.69KiB in 00:00:00 at 82.29KiB/s    


CompletedProcess(args=['yt-dlp', '-f', 'bestvideo+bestaudio/best', 'https://www.youtube.com/watch?v=HYSdFHQDDcs&t=30s'], returncode=0)

## part 1

In [9]:
import cv2
import numpy as np
from ultralytics import YOLO

class Detector:
    def __init__(self, model_path="model/yolov8n.pt", cls=2):
        self.model = YOLO(model_path)
        self.cls = cls

    def detect(self, frame):
        results = self.model(frame)
        return [(b[:4].cpu().numpy()) for r in results for b in r.boxes.data if int(b[5]) == self.cls]

class VideoProcessor:
    def __init__(self, input_path, output_path, detector):
        self.cap = cv2.VideoCapture(input_path)
        if not self.cap.isOpened():
            raise ValueError("Failed to open  file.")
        
        w, h, fps = int(self.cap.get(3)), int(self.cap.get(4)), int(self.cap.get(cv2.CAP_PROP_FPS))
        self.out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
        self.detector = detector

    def process(self):
        while self.cap.isOpened():
            ret, frame = self.cap.read()
            if not ret:
                break

            for x1, y1, x2, y2 in self.detector.detect(frame):
                c = ((x1 + x2) / 2, (y1 + y2) / 2)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                cv2.circle(frame, (int(c[0]), int(c[1])), 5, (0, 0, 255), -1)

            self.out.write(frame)

        self.cap.release()
        self.out.release()
        cv2.destroyAllWindows()

detector = Detector()
processor = VideoProcessor("data/video1.mp4", "output_video_part#1.mp4", detector)
processor.process()


0: 640x384 1 train, 53.7ms
Speed: 1.6ms preprocess, 53.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bus, 1 train, 41.1ms
Speed: 0.9ms preprocess, 41.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bus, 1 train, 43.4ms
Speed: 1.6ms preprocess, 43.4ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bus, 38.7ms
Speed: 1.0ms preprocess, 38.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bus, 60.4ms
Speed: 0.8ms preprocess, 60.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bus, 36.6ms
Speed: 0.8ms preprocess, 36.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bus, 39.3ms
Speed: 1.3ms preprocess, 39.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bus, 38.6ms
Speed: 1.3ms preprocess, 38.6ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 3

## part2

In [10]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from filterpy.kalman import KalmanFilter

class KalmanTracker:
    def __init__(self):
        self.kf = KalmanFilter(dim_x=4, dim_z=2)
        self.kf.F = np.eye(4) + np.array([[0, 0, 1, 0], [0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0]])
        self.kf.H = np.array([[1, 0, 0, 0], [0, 1, 0, 0]])
        self.kf.P *= 1000
        self.kf.R = np.eye(2) * 10
        self.kf.Q = np.eye(4) * 0.1

    def update(self, measurement):
        self.kf.update(measurement)
        self.kf.predict()
        return self.kf.x[:2]

class Detector:
    def __init__(self, model_path="model/yolov8n.pt", cls=2):
        self.model = YOLO(model_path)
        self.cls = cls

    def detect(self, frame):
        results = self.model(frame)
        return [(b[:4].cpu().numpy()) for r in results for b in r.boxes.data if int(b[5]) == self.cls]

class VideoProcessor:
    def __init__(self, input_path, output_path, detector, tracker):
        self.cap = cv2.VideoCapture(input_path)
        if not self.cap.isOpened():
            raise ValueError("Failed to open video file.")
        
        w, h, fps = int(self.cap.get(3)), int(self.cap.get(4)), int(self.cap.get(cv2.CAP_PROP_FPS))
        self.out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
        self.detector = detector
        self.tracker = tracker
        self.innovations = []

    def process(self):
        frame_idx = 0

        while self.cap.isOpened():
            ret, frame = self.cap.read()
            if not ret:
                break

            detections = self.detector.detect(frame)

            if detections:
                x1, y1, x2, y2 = detections[0]
                c = np.array([(x1 + x2) / 2, (y1 + y2) / 2])
            else:
                c = self.tracker.kf.x[:2]
                x1, y1, x2, y2 = int(c[0] - 20), int(c[1] - 20), int(c[0] + 20), int(c[1] + 20)

            pred = self.tracker.update(c)
            innovation = np.linalg.norm(c - pred)
            self.innovations.append((frame_idx, innovation))

            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.circle(frame, (int(pred[0]), int(pred[1])), 5, (0, 0, 255), -1)

            self.out.write(frame)
            frame_idx += 1

        self.cap.release()
        self.out.release()
        cv2.destroyAllWindows()

    def save_plot(self, plot_path):
        frames, magnitudes = zip(*self.innovations)
        plt.figure(figsize=(10, 5))
        plt.plot(frames, magnitudes, marker='o', linestyle='-', color='b')
        plt.xlabel("Frame")
        plt.ylabel("Innovation Magnitude")
        plt.title("Innovation Magnitude Over Time")
        plt.grid()
        plt.savefig(plot_path)
        plt.close()

detector = Detector()
tracker = KalmanTracker()
processor = VideoProcessor("data/video1.mp4", "output_video_part#2.mp4", detector, tracker)
processor.process()
processor.save_plot("output_plot_part#2.png")




0: 640x384 1 train, 66.9ms
Speed: 5.2ms preprocess, 66.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bus, 1 train, 41.0ms
Speed: 0.8ms preprocess, 41.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bus, 1 train, 37.5ms
Speed: 1.4ms preprocess, 37.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bus, 90.1ms
Speed: 0.7ms preprocess, 90.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)



/var/folders/pk/_6ypwpfj47j395tq1_jgdl600000gn/T/ipykernel_52520/2332105256.py:57: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x1, y1, x2, y2 = int(c[0] - 20), int(c[1] - 20), int(c[0] + 20), int(c[1] + 20)
/var/folders/pk/_6ypwpfj47j395tq1_jgdl600000gn/T/ipykernel_52520/2332105256.py:64: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.circle(frame, (int(pred[0]), int(pred[1])), 5, (0, 0, 255), -1)


0: 640x384 1 bus, 40.9ms
Speed: 0.9ms preprocess, 40.9ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bus, 45.2ms
Speed: 1.0ms preprocess, 45.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bus, 33.7ms
Speed: 0.8ms preprocess, 33.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bus, 32.6ms
Speed: 0.8ms preprocess, 32.6ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bus, 31.7ms
Speed: 1.3ms preprocess, 31.7ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bus, 36.0ms
Speed: 1.2ms preprocess, 36.0ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bus, 36.6ms
Speed: 1.7ms preprocess, 36.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bus, 1 truck, 34.3ms
Speed: 1.2ms preprocess, 34.3ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x

## part 3

In [11]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
import time

d = "cuda" if torch.cuda.is_available() else "cpu"
m = YOLO("model/yolo11n-obb.pt").to(d)

v = "data/video2.mp4"
c = cv2.VideoCapture(v)
if not c.isOpened():
    exit()

w = int(c.get(3))
h = int(c.get(4))
f = c.get(cv2.CAP_PROP_FPS)
t = int(c.get(cv2.CAP_PROP_FRAME_COUNT))
du = t / f if f > 0 else 1

out = cv2.VideoWriter("output_video_part#3.mp4", cv2.VideoWriter_fourcc(*"mp4v"), f, (w, h))

l2r = 0
r2l = 0
objs = {}
idc = 1
lx = w // 2
fs = 2
fc = 0
tr = {}
tl = 30
td = 3

def dtl(i, p, c, a):
    o = i.copy()
    for j in range(1, len(p)):
        cv2.line(o, p[j - 1], p[j], c, 2)
    cv2.addWeighted(o, a, i, 1 - a, 0, i)

while c.isOpened():
    r, f = c.read()
    if not r:
        break

    fc += 1
    if fc % fs != 0:
        continue

    ct = time.time()
    res = m(f, conf=0.1, device=d)
    if len(res) == 0 or not hasattr(res[0], 'obb') or res[0].obb is None:
        continue

    dts = res[0].obb.data.cpu().numpy() if res[0].obb is not None else np.array([])
    cur = {}
    for dt in dts:
        x, y, w_, h_, a, cf, cl = dt
        if int(cl) not in [9, 10]:
            continue

        cx = int(x)
        cy = int(y)

        oid = None
        for k, (px, py) in objs.items():
            if abs(px - cx) < w_ and abs(py - cy) < h_:
                oid = k
                break

        if oid is None:
            oid = idc
            idc += 1

        cur[oid] = (cx, cy)
        if oid in tr:
            tr[oid].append((cx, cy, ct))
            if len(tr[oid]) > tl:
                tr[oid].pop(0)
        else:
            tr[oid] = [(cx, cy, ct)]

        cv2.rectangle(f, (int(x - w_ / 2), int(y - h_ / 2)),
                      (int(x + w_ / 2), int(y + h_ / 2)), (0, 255, 0), 2)
        cv2.putText(f, f"ID: {oid}", (cx - 20, cy - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

        if oid in objs:
            px, _ = objs[oid]
            if px < lx and cx >= lx:
                l2r += 1
            elif px > lx and cx <= lx:
                r2l += 1

    objs = cur
    cv2.line(f, (lx, 0), (lx, h), (0, 255, 255), 2)

    for o, p in tr.items():
        fp = [(x, y) for (x, y, t) in p if ct - t <= td]
        dtl(f, fp, (200, 200, 200), 0.5)

    nl = l2r / du
    nr = r2l / du
    t1 = f"Right to Left: {r2l} (Norm: {nr:.2f})"
    t2 = f"Left to Right: {l2r} (Norm: {nl:.2f})"

    cv2.putText(f, t1, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(f, t2, (50, h - 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    out.write(f)

c.release()
out.release()
print(f"Final count - Left to Right: {l2r}, Right to Left: {r2l}")



0: 576x1024 103.2ms
Speed: 5.5ms preprocess, 103.2ms inference, 8.7ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 102.0ms
Speed: 3.8ms preprocess, 102.0ms inference, 1.1ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 82.9ms
Speed: 4.1ms preprocess, 82.9ms inference, 1.4ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 70.7ms
Speed: 3.0ms preprocess, 70.7ms inference, 1.2ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 73.4ms
Speed: 3.5ms preprocess, 73.4ms inference, 1.0ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 77.9ms
Speed: 3.4ms preprocess, 77.9ms inference, 1.5ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 73.2ms
Speed: 3.3ms preprocess, 73.2ms inference, 1.4ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 83.4ms
Speed: 8.6ms preprocess, 83.4ms inference, 1.1ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 75.9ms
Speed: 3.1ms preprocess, 75.9ms 